In [1]:
# %pip install -U lsdb dask nested-dask astropy light-curve
# %pip install 'matplotlib>=3.10'

In [2]:
from pathlib import Path

import light_curve as licu
import numpy as np
from lsdb import read_hats
from nested_pandas import NestedDtype

In [3]:
BANDS = "ugrizy"
# effective lsst band wavelengths, in angstroms, from 
# https://svo2.cab.inta-csic.es/svo/theory/fps3/index.php?mode=browse&gname=LSST&asttype=
band_wave_aa = {
    "u": 3751.20,
    "g": 4740.66,
    "r": 6172.34,
    "i": 7500.97,
    "z": 8678.90,
    "y": 9711.82,
}

In [4]:
release = 'v29_0_0_rc5'
hats_path = Path("/sdf/data/rubin/shared/lsdb_commissioning/hats") / release
# list dir
print(list(map(str, hats_path.iterdir())))

catalog_path = hats_path / "object_lc"

id_column = "objectId"
lc_column = "objectForcedSource"
psf_mag_columns = [f"{band}_psfMag" for band in BANDS]
input_feature_columns = psf_mag_columns
obj_lc = read_hats(
    catalog_path,
    columns=[id_column, "coord_ra", "coord_dec", lc_column] + input_feature_columns,
).map_partitions(
    lambda df: df.assign(
        lc=df[lc_column].astype(
                NestedDtype.from_pandas_arrow_dtype(df.dtypes[lc_column])
        ),
    ).drop(
        columns=[lc_column],
    ),
)
obj_lc

['/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0_rc5/dia_object', '/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0_rc5/object', '/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0_rc5/dia_object_lc_index', '/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0_rc5/object_lc_index', '/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0_rc5/dia_object_lc_x_ps1', '/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0_rc5/dia_source', '/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0_rc5/object_lc_x_ps1', '/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0_rc5/source', '/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0_rc5/dia_object_lc_5arcs', '/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0_rc5/object_lc_5arcs', '/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0_rc5/dia_object_lc', '/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0_rc5/object_forced_source', '/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0_rc5/dia_object_force

,objectId,coord_ra,coord_dec,u_psfMag,g_psfMag,r_psfMag,i_psfMag,z_psfMag,y_psfMag,lc
npartitions=39,,,,,,,,,,
"Order: 5, Pixel: 32",int64[pyarrow],double[pyarrow],double[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],"nested<parentObjectId: [int64], coord_ra: [dou..."
"Order: 6, Pixel: 136",...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
"Order: 4, Pixel: 2247",...,...,...,...,...,...,...,...,...,...
"Order: 3, Pixel: 562",...,...,...,...,...,...,...,...,...,...


In [5]:
obj_clean = obj_lc.dropna(
    subset=["lc.psfFlux", "lc.psfDiffFlux"],
).query(
    "~lc.psfFlux_flag"
    " and ~lc.psfDiffFlux_flag"
    " and ~lc.diff_PixelFlags_nodataCenter"
    " and ~lc.pixelFlags_bad"
    " and ~lc.pixelFlags_cr"
    " and ~lc.pixelFlags_crCenter"
    " and ~lc.pixelFlags_edge"
    " and ~lc.pixelFlags_interpolated"
    " and ~lc.pixelFlags_interpolatedCenter"
    " and ~lc.pixelFlags_nodata"
    " and ~lc.pixelFlags_saturated"
    " and ~lc.pixelFlags_saturatedCenter"
    " and ~lc.pixelFlags_suspect"
    " and ~lc.pixelFlags_suspectCenter"
    " and ~lc.invalidPsfFlag"
).dropna(
    subset="lc"
)

In [ ]:
flux_detection_extractor = licu.Extractor(
    licu.AndersonDarlingNormal(transform=True),
    licu.Duration(transform=True),
    licu.ObservationCount(transform=True),
    licu.Bins(
        [
            licu.Cusum(transform=True),
            licu.ObservationCount(transform=True),
            licu.ExcessVariance(transform=True),
            licu.Kurtosis(transform=True),
            licu.LinearFit(transform=True),
            licu.MeanVariance(transform=True),
            licu.Skew(transform=True),
        ],
        offset=0.0,
        window=1.0,
    )
)

magn_detection_extractor = licu.Extractor(
    licu.BeyondNStd(nstd=2.0, transform=True),
    licu.InterPercentileRange(0.05, transform=True),
    licu.InterPercentileRange(0.2, transform=True),
    licu.LinearTrend(transform=True),
    licu.MagnitudePercentageRatio(transform=True),
    licu.Median(transform=True),
    licu.OtsuSplit(),
    licu.PercentAmplitude(transform=True),
    licu.Periodogram(
        peaks=2,
        fast=False,
        nyquist='median',
        resolution=100,
        features=[
            licu.Amplitude(transform=True),
            licu.BeyondNStd(nstd=3.0, transform=True),
        ],
    ),
    licu.ReducedChi2(transform=True),
    licu.StetsonK(transform=True),
    licu.WeightedMean(transform=True),
)

flux_source_extractor = licu.Extractor(
    # licu.BazinFit("ceres", transform=True),
    licu.Duration(transform=True),
    licu.ObservationCount(transform=True),
    # licu.VillarFit("ceres", transform=True),
)

rainbow_direct = licu.RainbowFit.from_angstrom(band_wave_aa, with_baseline=True)
rainbow_dia = licu.RainbowFit.from_angstrom(band_wave_aa, with_baseline=False)


def extreact_features_one_band_detections(t, flux, fluxerr, *, fill_value: float, magnitudes: bool):
    assert t.ndim == flux.ndim == fluxerr.ndim == 1, f"Number of dimensions is {t.ndim = }, {flux.ndim = }, {fluxerr.ndim = } instead of 1"

    features = flux_detection_extractor(t, flux, fluxerr, fill_value=fill_value, sorted=True, check=True)
    names = np.array(flux_detection_extractor.names) + "_flux"
    
    if magnitudes:
        mag = -2.5 * np.log10(flux)
        magerr = 2.5 / np.log(np.array(10, dtype=np.float32)) * fluxerr / flux
        magn_features = magn_detection_extractor(t, mag, magerr, fill_value=fill_value, sorted=True, check=True)
        magn_names = np.array(magn_detection_extractor.names) + "_mag"

        features = np.concatenate([features, magn_features])
        names = np.concatenate([names, magn_names])

    return features, names


def extract_features_one_band_sources(t, flux, fluxerr, *, fill_value: float):
    features = flux_source_extractor(t, flux, fluxerr, sorted=True, check=True, fill_value=fill_value)
    names = np.array(flux_source_extractor.names)
    return features, names


def extract_features_one_band(t, flux, fluxerr, *, fill_value: float, magnitudes: bool):
    det_idx = flux > 5.0 * fluxerr
    det_features, det_names = extreact_features_one_band_detections(
        t[det_idx], flux[det_idx], fluxerr[det_idx], fill_value=fill_value, magnitudes=magnitudes
    )
    det_names = np.array(det_names) + "_det"
    
    src_features, src_names = extract_features_one_band_sources(t, flux, fluxerr, fill_value=fill_value)
    src_names = np.array(src_names) + "_src"

    return np.concatenate([det_features, src_features]), np.concatenate([det_names, src_names])


def extract_features(mjd, flux, flux_err, diff_flux, diff_flux_err, band, *, fill_value):
    t = np.asarray(mjd - 60_000.0, dtype=np.float32)

    features = []
    names = []

    assert t.ndim == flux.ndim == flux_err.ndim == diff_flux.ndim == diff_flux_err.ndim == 1, f"Number of dimensions is {t.ndim = }, {flux.ndim = }, {flux_err.ndim = }, {diff_flux.ndim = }, {diff_flux_err.ndim = } instead of 1"

    for b in BANDS:
        idx = band == b

        direct_band_features, direct_band_names = extract_features_one_band(t[idx], flux[idx], flux_err[idx], fill_value=fill_value, magnitudes=True)
        direct_band_names = direct_band_names + f'_{b}_direct'
        features.append(direct_band_features)
        names.append(direct_band_names)

        dia_band_features, dia_band_names = extract_features_one_band(t[idx], diff_flux[idx], diff_flux_err[idx], fill_value=fill_value, magnitudes=False)
        dia_band_names = dia_band_names + f'_{b}_dia'
        features.append(dia_band_features)
        names.append(dia_band_names)

    # direct_rainbow_features = rainbow_direct(t, flux, flux_err, band, fill_value=fill_value)
    # features.append(direct_rainbow_features)
    # names.append(np.array(rainbow_direct.names) + "_direct")

    # dia_rainbow_features = rainbow_dia(t, diff_flux, diff_flux_err, band, fill_value=fill_value)
    # features.append(dia_rainbow_features)
    # names.append(np.array(rainbow_dia.names) + "_dia")

    features = np.concatenate(features)
    names = np.concatenate(names)

    return dict(zip(names, features))


fill_value = -1.0
empty_float64 = np.array([], dtype=np.float64)
empty_float32 = np.array([], dtype=np.float32)
empty_band = np.array([], dtype=str)
empty_output = extract_features(empty_float64, empty_float32, empty_float32, empty_float32, empty_float32, empty_band, fill_value=fill_value)
lc_feature_columns = list(empty_output.keys())

obj_w_features = obj_clean.map_partitions(  # replace NaN magnitudes with 30.0
    lambda df: df.assign(**{col: df[col].fillna(30.0) for col in psf_mag_columns})
).sort_nested_values(
    "lc.midpointMjdTai",
).reduce(
    extract_features,
    "lc.midpointMjdTai",
    "lc.psfFlux",
    "lc.psfFluxErr",
    "lc.psfDiffFlux",
    "lc.psfDiffFluxErr",
    "lc.band",
    fill_value=fill_value,
    meta={name: np.float32 for name in lc_feature_columns},
    append_columns=True,
    infer_nesting=False,
)
obj_w_features

/sdf/home/k/kostya/rubin-user/linccf/.venv/lib/python3.12/site-packages/light_curve/light_curve_py/warnings.py:19: ExperimentalWarning: Function light_curve.light_curve_py.features.rainbow.generic.RainbowFit is experimental and may cause any kind of troubles
  warn_experimental(message)


,objectId,coord_ra,coord_dec,u_psfMag,g_psfMag,r_psfMag,i_psfMag,z_psfMag,y_psfMag,lc,lg_anderson_darling_normal_flux_det_u_direct,duration_flux_det_u_direct,observation_count_flux_det_u_direct,bins_window1.0_offset0.0_cusum_flux_det_u_direct,bins_window1.0_offset0.0_observation_count_flux_det_u_direct,bins_window1.0_offset0.0_excess_variance_flux_det_u_direct,bins_window1.0_offset0.0_arcsinh_kurtosis_flux_det_u_direct,bins_window1.0_offset0.0_linear_fit_slope_flux_det_u_direct,bins_window1.0_offset0.0_linear_fit_slope_sigma_flux_det_u_direct,bins_window1.0_offset0.0_linear_fit_reduced_chi2_flux_det_u_direct,bins_window1.0_offset0.0_mean_variance_flux_det_u_direct,bins_window1.0_offset0.0_arcsinh_skew_flux_det_u_direct,beyond_2_std_mag_det_u_direct,inter_percentile_range_5_mag_det_u_direct,inter_percentile_range_20_mag_det_u_direct,linear_trend_mag_det_u_direct,linear_trend_sigma_mag_det_u_direct,linear_trend_noise_mag_det_u_direct,magnitude_percentage_ratio_40_5_mag_det_u_direct,median_mag_det_u_direct,otsu_mean_diff_mag_det_u_direct,otsu_std_lower_mag_det_u_direct,otsu_std_upper_mag_det_u_direct,otsu_lower_to_all_ratio_mag_det_u_direct,percent_amplitude_mag_det_u_direct,period_0_mag_det_u_direct,period_s_to_n_0_mag_det_u_direct,period_1_mag_det_u_direct,period_s_to_n_1_mag_det_u_direct,periodogram_amplitude_mag_det_u_direct,periodogram_beyond_3_std_mag_det_u_direct,ln1p_chi2_mag_det_u_direct,stetson_K_mag_det_u_direct,weighted_mean_mag_det_u_direct,duration_src_u_direct,observation_count_src_u_direct,lg_anderson_darling_normal_flux_det_u_dia,duration_flux_det_u_dia,observation_count_flux_det_u_dia,bins_window1.0_offset0.0_cusum_flux_det_u_dia,bins_window1.0_offset0.0_observation_count_flux_det_u_dia,bins_window1.0_offset0.0_excess_variance_flux_det_u_dia,bins_window1.0_offset0.0_arcsinh_kurtosis_flux_det_u_dia,bins_window1.0_offset0.0_linear_fit_slope_flux_det_u_dia,bins_window1.0_offset0.0_linear_fit_slope_sigma_flux_det_u_dia,bins_window1.0_offset0.0_linear_fit_reduced_chi2_flux_det_u_dia,bins_window1.0_offset0.0_mean_variance_flux_det_u_dia,bins_window1.0_offset0.0_arcsinh_skew_flux_det_u_dia,duration_src_u_dia,observation_count_src_u_dia,lg_anderson_darling_normal_flux_det_g_direct,duration_flux_det_g_direct,observation_count_flux_det_g_direct,bins_window1.0_offset0.0_cusum_flux_det_g_direct,bins_window1.0_offset0.0_observation_count_flux_det_g_direct,bins_window1.0_offset0.0_excess_variance_flux_det_g_direct,bins_window1.0_offset0.0_arcsinh_kurtosis_flux_det_g_direct,bins_window1.0_offset0.0_linear_fit_slope_flux_det_g_direct,bins_window1.0_offset0.0_linear_fit_slope_sigma_flux_det_g_direct,bins_window1.0_offset0.0_linear_fit_reduced_chi2_flux_det_g_direct,bins_window1.0_offset0.0_mean_variance_flux_det_g_direct,bins_window1.0_offset0.0_arcsinh_skew_flux_det_g_direct,beyond_2_std_mag_det_g_direct,inter_percentile_range_5_mag_det_g_direct,inter_percentile_range_20_mag_det_g_direct,linear_trend_mag_det_g_direct,linear_trend_sigma_mag_det_g_direct,linear_trend_noise_mag_det_g_direct,magnitude_percentage_ratio_40_5_mag_det_g_direct,median_mag_det_g_direct,otsu_mean_diff_mag_det_g_direct,otsu_std_lower_mag_det_g_direct,otsu_std_upper_mag_det_g_direct,otsu_lower_to_all_ratio_mag_det_g_direct,percent_amplitude_mag_det_g_direct,period_0_mag_det_g_direct,period_s_to_n_0_mag_det_g_direct,period_1_mag_det_g_direct,period_s_to_n_1_mag_det_g_direct,periodogram_amplitude_mag_det_g_direct,periodogram_beyond_3_std_mag_det_g_direct,ln1p_chi2_mag_det_g_direct,stetson_K_mag_det_g_direct,weighted_mean_mag_det_g_direct,duration_src_g_direct,observation_count_src_g_direct,lg_anderson_darling_normal_flux_det_g_dia,duration_flux_det_g_dia,observation_count_flux_det_g_dia,bins_window1.0_offset0.0_cusum_flux_det_g_dia,bins_window1.0_offset0.0_observation_count_flux_det_g_dia,bins_window1.0_offset0.0_excess_variance_flux_det_g_dia,bins_window1.0_offset0.0_arcsinh_kurtosis_flux_det_g_dia,bins_window1.0_offset0.0_linear_fit_slope_flux_det_g_dia,bin

In [ ]:
from dask.distributed import Client

# Start with a small client
client = Client(n_workers=24, memory_limit="16GB", threads_per_worker=1)
client

/sdf/home/k/kostya/rubin-user/linccf/.venv/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 2245 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:2245/status,
Dashboard: http://127.0.0.1:2245/status,Workers: 24
Total threads: 24,Total memory: 357.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:18665,Workers: 24
Dashboard: http://127.0.0.1:2245/status,Total threads: 24
Started: Just now,Total memory: 357.63 GiB
Comm: tcp://127.0.0.1:6713,Total threads: 1
Dashboard: http://127.0.0.1:8619/status,Memory: 14.90 GiB
Nanny: tcp://127.0.0.1:14425,


In [ ]:
obj_w_features[[id_column] + input_feature_columns + lc_feature_columns].to_hats("object_lc_features")

2025-04-21 08:48:38,191 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 10.43 GiB -- Worker memory limit: 14.90 GiB
2025-04-21 09:26:16,741 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 10.43 GiB -- Worker memory limit: 14.90 GiB
2025-04-21 09:31:16,788 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 